In [2]:
## Add imports here
from collections import defaultdict
import re
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from collections import Counter
from tqdm.auto import tqdm
import pickle


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LEN = 256
print("Using device:", DEVICE)


class WordPieceTokenizer:
    def __init__(self):
        self.vocab = {"<PAD>", "<START>", "<STOP>", "<UNK>"}
        self.word2idx = {}
        self.split_words = [] # To store the current state of split words
        
    def preprocess_data(self, text: str) -> str:
        """ 
        Preprocess input text by:
        - speaker name and colon (e.g., "First Citizen :")
        - removing punctuations (: , . ? ! ;)
        - converting to lowercase
        """
        text = re.sub(r'^.*?:\s*', '', text) #remove speaker name and colon
        text = re.sub(r"[:,.?!;]", '', text) #remove punctuations
        return text.lower().strip()  #convert to lowercase and strip whitespace
    
    def split_word(self, word: str) -> List[str]:
        """ Split word into characters, adding ## prefix to non-initial characters """
        # WordPiece style splitting (similar to BPE Algorithm)
        if not word:
            return []
        chars = list(word)
        return [chars[0]] + [f"##{c}" for c in chars[1:]]
    
    def get_pair_frequencies(self, split_words: List[List[str]]) -> Dict[Tuple[str, str], int]:
        """ Count frequencies of adjacent pairs in the split words """
        pair_freqs = defaultdict(int)
        for word in split_words:
            for i in range(len(word) - 1):
                pair = (word[i], word[i + 1])
                pair_freqs[pair] += 1
        return pair_freqs
    
    def get_unit_frequencies(self, split_words: List[List[str]]) -> Dict[str, int]:
        """ Count frequencies of individual units """
        unit_freqs = defaultdict(int)
        for word in split_words:
            for unit in word:
                unit_freqs[unit] += 1
        return unit_freqs
    
    def calculate_pair_scores(self, pair_freqs: Dict[Tuple[str, str], int], 
                             unit_freqs: Dict[str, int]) -> Dict[Tuple[str, str], float]:
        """ Calculate scores for each pair using the formula: freq(pair) / (freq(first) * freq(second)) """
        scores = {}
        for pair, freq in pair_freqs.items():
            first, second = pair
            score = freq / (unit_freqs[first] * unit_freqs[second])
            scores[pair] = score
        return scores
    
    def merge_pair(self, pair: Tuple[str, str], split_words: List[List[str]]) -> List[List[str]]:
        """ 
        Merge the given subword pair in all words:
        - Locate adjacent occurrences of the pair in split words
        - Replace them with the merged token
        """
        first, second = pair
        merged = first + second.replace("##", "")
        
        new_split_words = []
        for word in split_words:
            new_word = []
            i = 0
            while i < len(word):
                if i < len(word) - 1 and word[i] == first and word[i + 1] == second:
                    new_word.append(merged)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_split_words.append(new_word)
        
        return new_split_words
    
    def read_corpus(self, file_path: str) -> List[str]:
        """ Read corpus from text file """
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.readlines()

    def construct_vocabulary(self, corpus_file: str, vocab_size: int) -> None:
        """ 
        Construct vocabulary using WordPiece algorithm:
        - Read and preprocess the corpus
        - Convert words into character-level subwords
        - Iteratively merge the most frequent subword pairs until reaching vocab_size using the scoring method
        - Save the vocabulary to a file 
        """
        # Read and preprocess corpus
        corpus = self.read_corpus(corpus_file)
        processed_texts = [self.preprocess_data(text) for text in corpus]
        
        # Split all words into characters with ## prefix
        self.split_words = []
        for text in processed_texts:
            for word in text.split():
                self.split_words.append(self.split_word(word))
        
        # Add initial units to vocabulary
        initial_units = set()
        for word in self.split_words:
            initial_units.update(word)
        self.vocab.update(initial_units)
        
        # Main loop for merging pairs until vocabulary size is reached
        pbar = tqdm(total=vocab_size - len(self.vocab), desc="Building vocabulary")
        while len(self.vocab) < vocab_size:
            # Get frequencies
            pair_freqs = self.get_pair_frequencies(self.split_words)
            if not pair_freqs:
                break
                
            unit_freqs = self.get_unit_frequencies(self.split_words)
            
            # Calculate scores
            scores = self.calculate_pair_scores(pair_freqs, unit_freqs)
            if not scores:
                break
                
            # Find best pair
            best_pair = max(scores.items(), key=lambda x: x[1])[0]
            merged_token = best_pair[0] + best_pair[1].replace("##", "")
            
            # Add to vocabulary and merge in corpus
            self.vocab.add(merged_token)
            self.split_words = self.merge_pair(best_pair, self.split_words)
            pbar.update(1)
        pbar.close()

        self.word2idx = {word: idx for idx, word in enumerate(self.vocab)}

        # Save vocabulary to file
        with open('vocabulary_35.txt', 'w', encoding='utf-8') as f:
            for token in sorted(self.vocab):
                f.write(f"{token}\n")
    
    def tokenize(self, text: str) -> List[str]:
        """ 
        Tokenize input text:
        - Preprocess the text
        - Convert each word into subwords
        - Merge the longest matching subwords found in the vocabulary
        - If no match is found, mark it as [UNK]
        """
        text = self.preprocess_data(text)
        result = []
        
        for word in text.split():
            # Character-level split
            current = self.split_word(word)
            
            # Merging using vocabulary as long as possible
            final_tokens = []
            i = 0
            while i < len(current):
                longest_match = None
                longest_length = 0
                
                # Try to find longest matching sequence starting at current position
                for j in range(i + 1, len(current) + 1):
                    candidate = current[i]
                    for k in range(i + 1, j):
                        candidate += current[k].replace("##", "")
                    if candidate in self.vocab and j - i > longest_length:
                        longest_match = candidate
                        longest_length = j - i
                
                if longest_match:
                    final_tokens.append(longest_match)
                    i += longest_length
                else:
                    # If no match found, mark as unknown
                    final_tokens.append("[UNK]")
                    i += 1
            
            result.extend(final_tokens)
        
        return result


def initialise_projections(dim_in, dim_out):
    """
    create projections for Q, K, V.
    """
    return (
        nn.Linear(dim_in, dim_out),
        nn.Linear(dim_in, dim_out),
        nn.Linear(dim_in, dim_out)
    )

def pairwise_similarities(query, key):
    """
    Compute dot product attention.
    """
    # batch_size x num_heads x seq_len x seq_len
    return torch.matmul(query, key.transpose(-2, -1))

def attention_scaled(similarities, dim_key):
    """
    Scale the raw attention scores.
    """
    return similarities / np.sqrt(dim_key) #scale by sqrt(d_k)

def attention_softmax(similarities):
    """
    Normalize the scaled raw attention scores with softmax.
    """
    return F.softmax(similarities, dim=-1) #softmax across the last dimension

def compute_outputs(attention_scores, values):
    """
    Get outputs as a weighted sum of values by attention scores.
    """
    return torch.matmul(attention_scores, values)

def make_causal_mask(seq_len, device=DEVICE):
    """
    Create a mask matrix that masks future context for the attention.
    """
    #lower triangular matrix with 1s
    mask = torch.tril(torch.ones(seq_len, seq_len, device=device))
    return mask.bool() #convert to boolean

def apply_causal_mask(similarities, mask):
    """
    Apply mask to attention.
    """
    #set masked postions (future tokens) to -inf
    masked_similarities = similarities.masked_fill(mask == 0, float("-inf"))
    return masked_similarities

def split_heads(x, num_heads, head_dim):
    """
    Splitting the input across multiple heads.
    """
    batch_size, seq_len, dim = x.size()
    #reshape to (batch_size, seq_len, num_heads, head_dim)
    x = x.view(batch_size, seq_len, num_heads, head_dim)
    #transpose to (batch_size, num_heads, seq_len, head_dim)
    return x.transpose(1, 2)

def merge_heads(x):
    """
    Reversing splitting action of function split_heads().
    """
    batch_size, num_heads, seq_len, head_dim = x.size()
    #transpose to (batch_size, seq_len, num_heads, head_dim)
    x = x.transpose(1, 2).contiguous()
    #reshape to (batch_size, seq_len, num_heads * head_dim)
    return x.view(batch_size, seq_len, num_heads * head_dim)

def self_attention(query, key, value, mask=None):
    """
    Self-attention block.
    """
    #compute raw attention scores
    similarities = pairwise_similarities(query, key)
    #scale the attention scores
    dim_key = key.size(-1)
    scaled_similarities = attention_scaled(similarities, dim_key)

    #apply mask if provided
    if mask is not None:
        scaled_similarities = apply_causal_mask(scaled_similarities, mask)
    
    #apply softmax to get attention scores
    attention = attention_softmax(scaled_similarities)
    #compute the output as a weighted sum of values
    outputs = compute_outputs(attention, value)

    return outputs, attention

def split_heads_qkv(query, key, value, num_heads, head_dim):
    """
    Split Q, K, V across multiple heads.
    """
    query = split_heads(query, num_heads, head_dim)
    key = split_heads(key, num_heads, head_dim)
    value = split_heads(value, num_heads, head_dim)
    return query, key, value

def load_and_preprocess_data():
    with open("shakespear_train.txt", "r") as f:
        lines_train = f.readlines()
    with open("shakespear_dev.txt", "r") as f:
        lines_dev = f.readlines()
    # with open("shakespear_test.txt", "r") as f:
    #     lines_test = f.readlines()

    tokens_train = [line.split() for line in lines_train]

    # Utility function to flatten tokens
    def flat(tokens):
        ## Your code here
        return [word for sentence in tokens for word in sentence]

    token_counts = Counter(flat(tokens_train))

    ## Create tokenizer
    tokenizer = pickle.load(open("tokenizer.pkl", 'rb'))

    ## Create inverse tokenizer for decoding
    tokenizer_inv = {idx: word for word, idx in tokenizer.word2idx.items()}

    ## Prepare datasets
    data_train = [line for line in lines_train]
    data_val = [line for line in lines_dev]

    ## Create input-output pairs
    train_dataset = data_train
    val_dataset = data_val

    return train_dataset, val_dataset, tokenizer, tokenizer_inv

def pad_to_length(tokens, max_len, tokenizer):
    """
    Pad tokens to a fixed length.
    """
    if len(tokens) >= max_len:
        return tokens[-max_len:]
    return tokens + [tokenizer["<PAD>"]] * (max_len - len(tokens))

def tokenize(sentence, pad_to_len=None, tokenizer=None, include_stop=True):
    """
    Tokenize a sentence.
    """
    if tokenizer is None:
        raise ValueError("Tokenizer is not provided.")
    
    tokens = tokenizer.tokenize(sentence)
    tokens = [tokenizer.word2idx.get(token, tokenizer.word2idx["<UNK>"]) for token in tokens]
    tokens = [tokenizer.word2idx["<START>"]] + tokens #add start token
    if include_stop:
        tokens += [tokenizer.word2idx["<STOP>"]] #add stop token
    if pad_to_len:
        tokens = pad_to_length(tokens, pad_to_len, tokenizer.word2idx) #pad if needed
    return tokens

def decode(tokens, tokenizer_inv, end_at_stop=True, omit_pad=True):
    """
    Decode tokens to text.
    """
    words = []
    for token in tokens:
        word = tokenizer_inv[token]
        if end_at_stop and word == "<STOP>":
            break #stop at STOP token if required
        if omit_pad and word == "<PAD>":
            continue #skip PAD tokens if required
        if word in ["<START>", "<STOP>", "<PAD>", "<UNK>"]:
            continue #skip special tokens
        if word[:2] == "##":
            if words:
                words[-1] += word[2:]
            else:
                words.append(word[2:])
        else:
            words.append(word)
    return " ".join(words)

@torch.no_grad()
def evaluate_losses(data, model, tokenizer, bs=32, progress=True, pad_to_len=MAX_LEN):
    it = range(0, len(data), bs)
    if progress:
        it = tqdm(it)

    out = []
    for b_start in it:
        batch = slice(b_start, b_start + bs)
        tokens = torch.tensor(
            [tokenize(t, pad_to_len=pad_to_len, tokenizer=tokenizer) for t in data[batch]], dtype=torch.long
        ).to(DEVICE)
        X_tokens, y_tokens = tokens[:, :-1].contiguous(), tokens[:, 1:].contiguous()

        model.eval()
        logits, _ = model(X_tokens)
        log_probs = F.log_softmax(logits, dim=-1)
        y_log_probs = torch.gather(log_probs, 2, y_tokens[..., None])[..., 0]

        for i in range(y_tokens.shape[0]):
            not_pad = y_tokens[i] != tokenizer.word2idx["<PAD>"]
            loss = -y_log_probs[i, not_pad].mean()
            out.append(loss.item())

    return out

def generate_text(model, tokenizer, tokenizer_inv, context="<START>", gen_tokens=10, temperature=0.6):
    """
    Generate a fixed number of tokens using the trained model.
    """
    ## Tokenize the context
    if context == "<START>":
        context_tokens = [tokenizer.word2idx["<START>"]]
    else:
        context_tokens = tokenize(context, tokenizer=tokenizer, include_stop=False)

    context_tensor = torch.tensor(context_tokens, dtype=torch.long).unsqueeze(0).to(DEVICE)

    model.eval()
    gen_tokens += 1
    with torch.no_grad():
        while gen_tokens:
            ## Get predictions
            logits, _ = model(context_tensor)

            ## Focus on the last token's predictions
            next_token_logits = logits[:, -1, :]

            ## Apply softmax to get probabilities
            if temperature == 0:
                # Greedy decoding
                next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)

            else:
                # Apply temperature scaling
                scaled_logits = next_token_logits / temperature
                probs = F.softmax(scaled_logits, dim=-1)
                
            ## Sample from the distribution
                next_token = torch.multinomial(probs, num_samples=1)

            ## Append to the context
            context_tensor = torch.cat((context_tensor, next_token), dim=1)

            if tokenizer_inv[next_token.item()][:2] != "##":
                gen_tokens -= 1 #reducing gen_tokens only when we get a full word

            ## Stop if we generated a STOP token
            if next_token.item() == tokenizer.word2idx["<STOP>"]:
                break

    ## Convert back to text
    generated_tokens = context_tensor[0].tolist()[:-1]
    return decode(generated_tokens, tokenizer_inv)

Using device: cuda


In [3]:
## Define the Transformer model
class TransformerLM(nn.Module):
    def __init__(self, vocab_size, d_model=512, nhead=8, num_layers=4, dim_feedforward=1024, dropout=0.1):        
        super(TransformerLM, self).__init__()

        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)

        #create positional encoding
        pos_encoding = torch.zeros(1, MAX_LEN, d_model)
        position = torch.arange(MAX_LEN).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
        #fill positional encoding tensor
        pos_encoding[0, :, 0::2] = torch.sin(position * div_term)
        pos_encoding[0, :, 1::2] = torch.cos(position * div_term)
        #register as buffer (persistent state but not a parameter)
        self.register_buffer('pos_encoding', pos_encoding)

        self.blocks = nn.ModuleList([
            TransformerBlock(d_model, nhead, dim_feedforward, dropout)
            for _ in range(num_layers)]) #create transformer blocks

        self.output_layer = nn.Linear(d_model, vocab_size) #final linear layer for token prediction
        self._init_weights() #initialize weights

    def _init_weights(self):
        nn.init.normal_(self.embedding.weight, mean=0.0, std=0.02)
        nn.init.normal_(self.output_layer.weight, mean=0.0, std=0.02)
        nn.init.zeros_(self.output_layer.bias)

    def forward(self, x):
        seq_len = x.size(1) #x has shape [batch_size, seq_len]
        #get token embeddings
        embeddings = self.embedding(x)  # [batch_size, seq_len, d_model]
        #add positional encoding
        x = embeddings + self.pos_encoding[:, :seq_len, :]
        #create causal mask
        mask = make_causal_mask(seq_len)

        #apply transformer blocks
        attentions = []
        for block in self.blocks:
            x, attn = block(x, mask)
            attentions.append(attn)
        #project to vocabulary
        logits = self.output_layer(x)
        return logits, attentions


class TransformerBlock(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout):
        super(TransformerBlock, self).__init__()

        self.attention = MultiHeadAttention(d_model, nhead, dropout)
        #layer normalization and dropout
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

        #mLP block
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.GELU(),
            nn.Linear(dim_feedforward, d_model),
            nn.Dropout(dropout))

    def forward(self, x, mask=None):
        #self-attention block with residual connection
        attn_output, attn_weights = self.attention(x, x, x, mask)
        x = x + self.dropout(attn_output)
        x = self.norm1(x)

        #feed-forward block with residual connection
        ff_output = self.feed_forward(x)
        x = x + self.dropout(ff_output)
        x = self.norm2(x)

        return x, attn_weights


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.1):
        super(MultiHeadAttention, self).__init__()

        self.d_model = d_model
        self.nhead = nhead
        self.head_dim = d_model // nhead
        assert self.head_dim * nhead == d_model, "d_model must be divisible by nhead"

        #linear projections for Q, K, V
        self.q_proj, self.k_proj, self.v_proj = initialise_projections(d_model, d_model)

        self.out_proj = nn.Linear(d_model, d_model) #output projection
        self.dropout = nn.Dropout(dropout) #dropout layer

    def forward(self, query, key, value, mask=None):
        #linear projections
        q = self.q_proj(query)
        k = self.k_proj(key)
        v = self.v_proj(value)

        q, k, v = split_heads_qkv(q, k, v, self.nhead, self.head_dim) #split heads
        attn_output, attn_weights = self_attention(q, k, v, mask) #self-attention
        attn_output = merge_heads(attn_output) #merge heads
        output = self.out_proj(attn_output) #output projection

        return output, attn_weights


## Training function
def train_model(model, train_loader, val_dataset, tokenizer, tokenizer_inv, epochs=10, lr=1e-4):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)
    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.word2idx["<PAD>"])

    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        #training on train set
        model.train()
        epoch_loss = 0.0

        for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")):
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            outputs, _ = model(inputs)

            #reshape for CrossEntropyLoss
            outputs = outputs.view(-1, outputs.size(-1))
            targets = targets.view(-1)
            #find loss and backpropagate
            loss = criterion(outputs, targets)
            loss.backward()
            #gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            epoch_loss += loss.item()
        
        #calculate average training loss
        avg_train_loss = epoch_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        #evaluate on validation set
        model.eval()
        with torch.no_grad():
            val_losses_epoch = evaluate_losses(val_dataset, model, tokenizer)
            avg_val_loss = sum(val_losses_epoch) / len(val_losses_epoch)
            val_losses.append(avg_val_loss)
            
            #update learning rate
            scheduler.step(avg_val_loss)

        print(f"Epoch {epoch+1} | Loss: Train= {avg_train_loss:.4f}, Val= {avg_val_loss:.4f} | Perplexity: Train= {np.exp(avg_train_loss):.2f}, Val= {np.exp(avg_val_loss):.2f}")

        ## Generate a sample text
        sample_text = generate_text(model, tokenizer, tokenizer_inv)
        print(f"Sample text: {sample_text}\n")

    return train_losses, val_losses

In [3]:
if __name__ == "__main__":
    tokenizer = WordPieceTokenizer()
    tokenizer.construct_vocabulary('shakespear_train.txt', vocab_size=10000)
    with open('tokenizer.pkl', 'wb') as f:
        pickle.dump(tokenizer, f)

Building vocabulary: 100%|██████████| 9939/9939 [1:03:59<00:00,  2.59it/s]


Vocabulary size: 10000
Train samples: 9837
Validation samples: 1304

Model architecture:
- Embedding dimension: 512
- Number of heads: 8
- Number of layers: 6
- Feedforward dimension: 1024
- Total parameters: 22866704



100%|██████████| 41/41 [00:01<00:00, 32.01it/s]


Epoch 1 | Loss: Train= 4.6722, Val= 4.1607 | Perplexity: Train= 106.93, Val= 64.12
Sample text: what leres ir



100%|██████████| 41/41 [00:05<00:00,  7.28it/s]


Epoch 2 | Loss: Train= 3.9862, Val= 4.0166 | Perplexity: Train= 53.85, Val= 55.51
Sample text: werer rertt beereden thes



100%|██████████| 41/41 [00:01<00:00, 29.10it/s]


Epoch 3 | Loss: Train= 3.8500, Val= 3.9291 | Perplexity: Train= 46.99, Val= 50.86
Sample text: thesener and thatered theat in mereren sanerune rarere



100%|██████████| 41/41 [00:05<00:00,  7.21it/s]


Epoch 4 | Loss: Train= 3.7305, Val= 3.8065 | Perplexity: Train= 41.70, Val= 44.99
Sample text: hener 'd theen therersede and of theis



100%|██████████| 41/41 [00:00<00:00, 67.28it/s]


Epoch 5 | Loss: Train= 3.6171, Val= 3.6999 | Perplexity: Train= 37.23, Val= 40.44
Sample text: heredes



100%|██████████| 41/41 [00:05<00:00,  7.27it/s]


Epoch 6 | Loss: Train= 3.5196, Val= 3.6392 | Perplexity: Train= 33.77, Val= 38.06
Sample text: thou your my lord



100%|██████████| 41/41 [00:00<00:00, 67.28it/s]


Epoch 7 | Loss: Train= 3.4375, Val= 3.5795 | Perplexity: Train= 31.11, Val= 35.85
Sample text: it thee my lord



100%|██████████| 41/41 [00:05<00:00,  7.38it/s]


Epoch 8 | Loss: Train= 3.3662, Val= 3.5244 | Perplexity: Train= 28.97, Val= 33.93
Sample text: nost thou let when it therefore your to that e'ers



100%|██████████| 41/41 [00:05<00:00,  7.23it/s]


Epoch 9 | Loss: Train= 3.2951, Val= 3.4843 | Perplexity: Train= 26.98, Val= 32.60
Sample text: o heaven brother



100%|██████████| 41/41 [00:01<00:00, 29.14it/s]


Epoch 10 | Loss: Train= 3.2373, Val= 3.4429 | Perplexity: Train= 25.46, Val= 31.28
Sample text: in the second as i will beare my lord



100%|██████████| 41/41 [00:05<00:00,  7.11it/s]


Epoch 11 | Loss: Train= 3.1761, Val= 3.4329 | Perplexity: Train= 23.95, Val= 30.97
Sample text: o the resterly and the reserous breate gore aty for



100%|██████████| 41/41 [00:01<00:00, 30.10it/s]


Epoch 12 | Loss: Train= 3.1218, Val= 3.4134 | Perplexity: Train= 22.69, Val= 30.37
Sample text: can not then in the time of the sentters of



100%|██████████| 41/41 [00:05<00:00,  7.40it/s]


Epoch 13 | Loss: Train= 3.0685, Val= 3.3937 | Perplexity: Train= 21.51, Val= 29.78
Sample text: heaven none and these the kings of they duke of



100%|██████████| 41/41 [00:00<00:00, 62.58it/s]


Epoch 14 | Loss: Train= 3.0161, Val= 3.3712 | Perplexity: Train= 20.41, Val= 29.11
Sample text: they beaun at of an despaty



100%|██████████| 41/41 [00:00<00:00, 67.19it/s]


Epoch 15 | Loss: Train= 2.9702, Val= 3.3560 | Perplexity: Train= 19.50, Val= 28.67
Sample text: but i pray thee



100%|██████████| 41/41 [00:00<00:00, 67.14it/s]


Epoch 16 | Loss: Train= 2.9229, Val= 3.3448 | Perplexity: Train= 18.60, Val= 28.36
Sample text: then is the name of seal and thy day were



100%|██████████| 41/41 [00:00<00:00, 67.27it/s]


Epoch 17 | Loss: Train= 2.8742, Val= 3.3257 | Perplexity: Train= 17.71, Val= 27.82
Sample text: now my lord is this gentlemans



100%|██████████| 41/41 [00:00<00:00, 67.29it/s]


Epoch 18 | Loss: Train= 2.8339, Val= 3.3214 | Perplexity: Train= 17.01, Val= 27.70
Sample text: hadst thou haste thy beent



100%|██████████| 41/41 [00:00<00:00, 67.24it/s]


Epoch 19 | Loss: Train= 2.7877, Val= 3.3072 | Perplexity: Train= 16.24, Val= 27.31
Sample text: ay



100%|██████████| 41/41 [00:00<00:00, 67.16it/s]


Epoch 20 | Loss: Train= 2.7431, Val= 3.3278 | Perplexity: Train= 15.54, Val= 27.88
Sample text: why then is the king



In [4]:
def main():
    ## Load and preprocess data
    train_dataset, val_dataset, tokenizer, tokenizer_inv = load_and_preprocess_data()
    print(f"Vocabulary size: {len(tokenizer.vocab)}")
    print(f"Train samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}\n")

    ## Create data loaders
    def collate_fn(batch):
        #tokenize and pad
        tokens = [torch.tensor(tokenize(text, pad_to_len=MAX_LEN, tokenizer=tokenizer)) for text in batch]
        tokens = torch.stack(tokens)
        
        #create input-target pairs (inputs are tokens[:-1], targets are tokens[1:])
        inputs = tokens[:, :-1].contiguous()
        targets = tokens[:, 1:].contiguous()
        
        return inputs, targets
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

    ## Model hyperparameters
    vocab_size = len(tokenizer.vocab)
    d_model = 512
    nhead = 8
    num_layers = 6
    dim_feedforward = 1024
    dropout = 0.1

    ## Initialize model
    model = TransformerLM(
        vocab_size=vocab_size,
        d_model=d_model,
        nhead=nhead,
        num_layers=num_layers,
        dim_feedforward=dim_feedforward,
        dropout=dropout
    ).to(DEVICE)

    ## Print model summary
    print(f"Model architecture:")
    print(f"- Embedding dimension: {d_model}")
    print(f"- Number of heads: {nhead}")
    print(f"- Number of layers: {num_layers}")
    print(f"- Feedforward dimension: {dim_feedforward}")
    print(f"- Total parameters: {sum(p.numel() for p in model.parameters())}\n")

    ## Train the model
    train_losses, val_losses = train_model(
        model=model,
        train_loader=train_loader,
        val_dataset=val_dataset,
        tokenizer=tokenizer,
        tokenizer_inv=tokenizer_inv,
        epochs=20,
        lr=1e-4
    )

    ## Plot training and validation losses
    epochs = range(1, len(train_losses)+1)
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_losses, label='Training Loss')
    plt.plot(epochs, val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig('loss-plot.png')
    plt.show()

    ## Save the model
    torch.save({
        'model_state_dict': model.state_dict(),
        'tokenizer': tokenizer,
        'tokenizer_inv': tokenizer_inv,
        'config': {
            'd_model': d_model,
            'nhead': nhead,
            'num_layers': num_layers,
            'dim_feedforward': dim_feedforward,
            'dropout': dropout
        }
    }, 'task1-transformer.pt')

    ## Evaluate on test data
    # with open("/content/shakespear_test.txt", "r") as f:
    #     lines_test = f.readlines()

    # print(f"\nTest perplexity: {}")

if __name__ == "__main__":
    main()

In [5]:
def inference(model_obj, test_file, tokenizer, tokenizer_inv, gen_tokens=10, temperature=0.6):
    ## Load the saved model
    config = model_obj['config']
    model = TransformerLM(
        vocab_size=len(tokenizer_inv),
        d_model=config['d_model'],
        nhead=config['nhead'],
        num_layers=config['num_layers'],
        dim_feedforward=config['dim_feedforward'],
        dropout=config['dropout']
    ).to(DEVICE)
    model.load_state_dict(model_obj['model_state_dict'])
    model.eval()

    ## Read and process the input from test.txt
    with open(test_file, "r") as f:
        test_dataset = [line.strip() for line in f.readlines()]

    ## Generate text and calculate perplexity
    generated_text = []
    with torch.no_grad():
        for line in test_dataset:
            text = generate_text(model, tokenizer, tokenizer_inv, context=line, gen_tokens=gen_tokens)
            generated_text.append(text)
        test_loss = evaluate_losses(test_dataset, model, tokenizer)
        avg_test_loss = sum(test_loss) / len(test_loss)
        test_perplexity = np.exp(avg_test_loss)
    return generated_text, test_perplexity


model_path = 'task1-transformer.pt'
test_file = 'sample-test.txt'

model_obj = torch.load(model_path, map_location=torch.device('cpu'), weights_only=False)
tokenizer = model_obj['tokenizer']
tokenizer_inv = model_obj['tokenizer_inv']
generated_texts, ppl = inference(model_obj, test_file, tokenizer, tokenizer_inv)

## Print the generated text and perplexity
print(f"\nTest perplexity: {ppl:.2f}\n")
print("Generated Text:")
for text in generated_texts:
    print(text)

100%|██████████| 1/1 [00:00<00:00, 234.27it/s]


Test perplexity: 49.45

Generated Text:
well then imprison him if imprisonment be there in their power 's heart 's heart
i will pray pompey to them to thee hear thee
it was a mad fantastical therefore in whoermiesters of some foresself
ere he would have hanged thee that i have in the body 's atterdies in
either this is the envy
